In [1]:
import tensorflow
tensorflow.config.experimental.set_visible_devices([], 'GPU')

In [2]:
SEED = 42
from random import seed as py_seed
py_seed(SEED)
from numpy.random import seed as np_seed
np_seed(SEED)
from tensorflow import random as tf_random
tf_random.set_seed(SEED)

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD, RMSprop
from keras.utils import np_utils
from keras.models import load_model
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.nasnet import NASNetMobile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import random
from sklearn.model_selection import KFold, train_test_split


Using TensorFlow backend.


In [4]:
tensorflow.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# Data comes from here. This should contain a bunch of folders of classes, each containing examples
DATA_PATH = Path('C:/Users/kheen/Downloads/food-101/food-101/images')

# Get class (folder) names
classes = sorted([entry.name for entry in DATA_PATH.iterdir() if entry.is_dir()])

label_list = classes
labels = []
for i in range(50):
    num =  random.randint(0, (100 - i))
    labels.append(label_list[num])
    label_list.pop(num)
print(labels)
classes = labels
        
# Load list of filenames and classes, and convert to numpy array to allow array slicing 
# For k-fold validation
dataset = np.array([
    (f"{clazz}/{entry.name}", clazz)
    for clazz in classes
    for entry in (DATA_PATH / clazz).iterdir()
])

['ramen', 'carrot_cake', 'beef_carpaccio', 'takoyaki', 'filet_mignon', 'edamame', 'deviled_eggs', 'chicken_quesadilla', 'ceviche', 'strawberry_shortcake', 'pizza', 'cannoli', 'samosa', 'macaroni_and_cheese', 'beet_salad', 'beef_tartare', 'cheesecake', 'fish_and_chips', 'french_onion_soup', 'prime_rib', 'tacos', 'beignets', 'spaghetti_bolognese', 'falafel', 'shrimp_and_grits', 'oysters', 'fried_calamari', 'pho', 'guacamole', 'apple_pie', 'donuts', 'pork_chop', 'macarons', 'hot_and_sour_soup', 'cup_cakes', 'garlic_bread', 'omelette', 'churros', 'bruschetta', 'frozen_yogurt', 'caprese_salad', 'fried_rice', 'steak', 'gnocchi', 'pad_thai', 'dumplings', 'sushi', 'bibimbap', 'scallops', 'lasagna']


In [6]:
# create data generators

# Rescale 0-255 to 0-1
RESCALE=1./255

# This type of data generator is used to train the model
train_datagen = ImageDataGenerator(
    rescale=RESCALE,
    # Factors used to control
    rotation_range=45,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.125,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.125,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images

)

# This type of generator is used to test the model
test_datagen = ImageDataGenerator(
    rescale=RESCALE,
)



In [7]:
def create_model(input_shape, num_classes):
    # Base model, with weights pre-trained on ImageNet.
    base_model = NASNetMobile(input_shape, weights='imagenet', include_top=False)

    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    optimizer = RMSprop()

    model.compile(
        loss = 'categorical_crossentropy',
        optimizer = optimizer,
        metrics = ['accuracy']
    )
    
    return model

In [ ]:
# Image data generator settings
TARGET_SIZE = (224,224)
COLOR_MODE = "rgb"
BATCH_SIZE = 64
CLASS_MODE = "categorical"


# Training settings
NUM_FOLDS = 2
VALIDATION_SPLIT = 0.1
NUM_EPOCHS = 7

# Model settings


kf = KFold(n_splits=NUM_FOLDS, random_state=SEED, shuffle=True)
results = []
cur_fold = 1

for train_index, test_index in kf.split(dataset):
    # Print a message
    print(f"Fold {cur_fold} of {NUM_FOLDS}")
    cur_fold += 1
    
    # Extract train dataset
    d_train = dataset[train_index]
    # Split train dataset to form train and validation dataset
    d_train, d_val = train_test_split(dataset, test_size = VALIDATION_SPLIT)
    
    # These are used to convert the filename,class pairs to dataframes
    FILENAME_COL = "filename"
    CLASS_COL = "class"
    COLUMNS = [FILENAME_COL, CLASS_COL]
    # Convert to dataframes
    d_train = pd.DataFrame(d_train, columns = COLUMNS)
    d_val = pd.DataFrame(d_val, columns = COLUMNS)
    
    # Create training data generators
    # Training image data generator
    train_it = train_datagen.flow_from_dataframe(
        d_train,
        directory = DATA_PATH,
        x_col = FILENAME_COL,
        y_col = CLASS_COL,
        target_size = TARGET_SIZE,
        color_mode = COLOR_MODE,
        batch_size = BATCH_SIZE,
        class_mode = CLASS_MODE,
        shuffle = True,
        seed = SEED
    )
    # Validation image data generator
    val_it = test_datagen.flow_from_dataframe(
        d_val,
        directory = DATA_PATH,
        x_col = FILENAME_COL,
        y_col = CLASS_COL,
        target_size = TARGET_SIZE,
        color_mode = COLOR_MODE,
        batch_size = BATCH_SIZE,
        class_mode = CLASS_MODE,
        shuffle = False,
    )
    
    # Perform training
    # Build the model
    input_shape = (*TARGET_SIZE,3)
    num_classes = len(classes)
    model = create_model(input_shape, num_classes)
    
    # Train the model
    history = model.fit(
        train_it,
        validation_data=val_it,
        epochs=NUM_EPOCHS,
        workers=6,
        max_queue_size=100,
        verbose=True
    )
    
        
    # save model
    model.save("food_model_50.h5")
    
    # Extract test dataset and convert to dataframe
    d_test = pd.DataFrame(dataset[test_index], columns = COLUMNS)
    # Create test image data generator
    test_it = test_datagen.flow_from_dataframe(
        d_test,
        directory = DATA_PATH,
        x_col = FILENAME_COL,
        y_col = CLASS_COL,
        target_size = TARGET_SIZE,
        color_mode = COLOR_MODE,
        batch_size = BATCH_SIZE,
        class_mode = CLASS_MODE,
        shuffle = False,
    )
    # Test
    fold_results = model.evaluate(test_it)
    results.append(fold_results)
    print(f"Results for current fold: {fold_results}")

Fold 1 of 2
Found 45000 validated image filenames belonging to 50 classes.
Found 5000 validated image filenames belonging to 50 classes.
Epoch 1/7
704/704 [==============================] - 5505s 8s/step - loss: 2.7781 - accuracy: 0.2880 - val_loss: 1.5261 - val_accuracy: 0.3820
Epoch 2/7
704/704 [==============================] - 3250s 5s/step - loss: 2.2710 - accuracy: 0.3980 - val_loss: 1.1101 - val_accuracy: 0.4012
Epoch 3/7
704/704 [==============================] - 3509s 5s/step - loss: 2.1860 - accuracy: 0.4234 - val_loss: 1.2156 - val_accuracy: 0.4370
Epoch 4/7
704/704 [==============================] - 3213s 5s/step - loss: 2.1549 - accuracy: 0.4323 - val_loss: 1.0794 - val_accuracy: 0.4376
Epoch 5/7
704/704 [==============================] - 3264s 5s/step - loss: 2.1428 - accuracy: 0.4342 - val_loss: 1.1453 - val_accuracy: 0.4404
Epoch 6/7
704/704 [==============================] - 3332s 5s/step - loss: 2.1365 - accuracy: 0.4402 - val_loss: 1.1549 - val_accuracy: 0.4438
Epoch